# 拓扑排序介绍

## 基本概念

在图论中，拓扑排序（Topological Sorting）是一个有向无环图（DAG, Directed Acyclic Graph）的所有顶点的线性序列。  
且该序列必须满足下面两个条件：

1. 每个顶点出现且只出现一次。
2. 若存在一条从顶点 A 到顶点 B 的路径，那么在序列中顶点 A 出现在顶点 B 的前面。

注：有向无环图（DAG）才有拓扑排序，非DAG图没有拓扑排序一说。

> 有向无环图的定义为：如果一个有向图从任意顶点出发无法经过若干条边回到这个点，则称这个图是一个有向无环图。

## 实现原理

### Kahn's algorithm

该方法每次从拓扑图中删去入度为0的顶点，并删除与其相关的边，直到无入度为0的顶点存在。由于拓扑图的环路中不存在入度为零的点，因此该方法可以自动识别有环图。为了快速的计算每个顶点的入度，算法需要一个辅助数据来记录拓扑图中每个顶点的入度。  
算法的具体实现流程如下：

1. 从DAG图中选择一个没有前驱（即入度为0）的顶点并输出。
2. 从图中删除该顶点和所有以它为起点的有向边。
3. 重复步骤1和2直到当前的DAG图为空或当前图中不存在无前驱的顶点为止。

**时间复杂度**：遍历拓扑图中的每个顶点和边，因此复杂度为 $O(n+v)$, 其中 $n$ 代表拓扑图中顶点的数目，$v$ 代表拓扑图中边的数目。

**空间复杂度**：需要辅助数据记录每个顶点的入度，以及堆栈来临时保存入度为0的顶点，因此复杂度为 $O(n)$ 。

In [ ]:
# Kahn's algorithm实现
import algviz

def topologicalSortingKahns(graph):
    # 计算每个顶点的入度。
    viz = algviz.Visualizer(3.0)
    graph_viz = viz.createGraph(data=list(graph.values()), horizontal=True)
    degree = [0 for _ in range(len(graph))]
    for u in graph.values():
        for v in u.neighbors:
            degree[v.val] += 1
    que = viz.createVector(name='Queue')
    # 将入度为0的点放入队列。
    for u in graph.values():
        if degree[u.val] == 0:
            que.append(u)
    # 取点，删边，加点。
    cur_node = viz.createGraphTrace(graph_viz, hold=True)
    res = list()
    while len(que) > 0:
        cur_node(que.pop())
        for i in range(len(cur_node)):
            v = cur_node[i]
            degree[v.val] -= 1
            if degree[v.val] == 0:
                que.append(v)
        res.append(cur_node.val)
        viz.refresh()
    return res

### 深度优先搜索：

在使用DFS的方式遍历图时，可以按照一定的顺序将图中的顶点打印出来，为了实现打印出来顶点的拓扑排序，需要在一个顶点的邻居点之前打印它。该算法与传统的DFS方法的不同之处在于：**DFS拓扑排序是先递归后访问节点，而传统的DFS是先访问节点后递归。**  

下面的代码对分别实现了基于深度优先搜索的拓扑排序和传统的深搜算法：

In [ ]:
# 基于深度优先搜索的拓扑排序算法实现（递归版本）。
class Solution:
    def topologicalSortingDfsUnit(self, node, visited, stack):
        visited[node.val] = True
        self.trace(node)
        self.viz.refresh()
        for i in range(len(self.trace)):
            v = self.trace[i]
            if not visited[v.val]:
                self.topologicalSortingDfsUnit(v, visited, stack)
        stack.append(node.val)

    def topologicalSortingDfs(self, graph):
        self.viz = algviz.Visualizer(3.0)
        graph_viz = self.viz.createGraph(data=list(graph.values()), horizontal=True)
        self.trace = self.viz.createGraphTrace(graph_viz, hold=True)
        visited = [False for _ in range(len(graph))]
        stack = list()
        for u in graph.values():
            if not visited[u.val]:
                self.topologicalSortingDfsUnit(u, visited, stack)
        return stack[::-1]

In [ ]:
# 传统的拓扑图的深度优先搜索算法（递归版本）。
def graphDfsUnit(graph, u, visited, stack):
    visited[u] = True
    stack.append(u)
    for v in graph.adjtab[u]:
        if not visited[v]:
            graphDfsUnit(graph, v, visited, stack)
            
def graphDfs(graph):
    n = len(graph.adjtab)
    visited = [False for _ in range(n)]
    stack = list()
    for u in range(n):
        if not visited[u]:
            graphDfsUnit(graph, u, visited, stack)
    return stack

In [ ]:
graph1 = algviz.parseGraph('[[0,0], [1,1], [2,2], [3,3], [4,4]]',
                          '[[0,1], [0,3], [1,3], [1,2], [3,2], [3,4], [2,4]]')
#print("Call Kahn's topological sorting algorithm:")
#print("Result:", topologicalSortingKahns(graph1))
print('Call DFS topological sorting algorithm:')
%pdb
print('Result:', Solution().topologicalSortingDfs(graph1))

In [ ]:
graph2 = DirectedGraph(6)
graph2.add_edge(4,0)
graph2.add_edge(5,0)
graph2.add_edge(5,2)
graph2.add_edge(3,1)
graph2.add_edge(2,3)
graph2.add_edge(4,1)
print('Kanhs topologic sorting:', topologicalSortingKahns(graph2))
print('DFS topologic sorting:', topologicalSortingDfs(graph2))
print('DFS traverse:', graphDfs(graph2))

In [ ]:
# 有环图的处理。
graph3 = DirectedGraph(3)
graph3.add_edge(0,1)
graph3.add_edge(1,2)
graph3.add_edge(2,1)
# Kanhs算法可以处理有环图。
print('Kanhs topologic sorting:', topologicalSortingKahns(graph3))
# DFS对于有环图排出来的结果是错误的。
print('DFS topologic sorting:', topologicalSortingDfs(graph3))

## 参考链接

+ https://zhuanlan.zhihu.com/p/56024487
+ https://songlee24.github.io/2015/05/07/topological-sorting/
+ https://www.geeksforgeeks.org/topological-sorting/